In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import os 
if os.path.exists("PRBNet_PyTorch"):
  print("PRBNet code file had been cloned")
else:
  !git clone https://github.com/pingyang1117/PRBNet_PyTorch.git
%cd PRBNet_PyTorch 
!ls

PRBNet code file had been cloned
/content/drive/MyDrive/PRBNet_PyTorch
assets	datasets  LICENSE  obb	prb  README.md


In [ ]:
%cd /content/drive/MyDrive/PRBNet_PyTorch_Final/prb

/content/drive/MyDrive/PRBNet_PyTorch_Final/prb


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 39.4 MB/s 


##Re-parameterization

In [ ]:
# import
from copy import deepcopy
from models.yolo import Model
import torch
from utils.torch_utils import select_device, is_parallel
import yaml

cfg_path = r"cfg/deploy/yolov7-PRB.yaml"
ckpt_path = r"/content/drive/MyDrive/PRBNet_PyTorch/prb/yolov7-prb_training.pt" # checkpoints of yolov7-prb with IDetect layer
save_path = r"yolov7-prb_training.pt"  # output file path

device = select_device('0', batch_size=1)
# model trained by cfg/training/*.yaml
ckpt = torch.load(ckpt_path, map_location=device)
# reparameterized model in cfg/deploy/*.yaml
model = Model(cfg_path, ch=3, nc=80).to(device)

with open(cfg_path) as f:
    yml = yaml.load(f, Loader=yaml.SafeLoader)
anchors = len(yml['anchors'])

# copy intersect weights
state_dict = ckpt['model'].float().state_dict()
exclude = []
intersect_state_dict = {k: v for k, v in state_dict.items() if k in model.state_dict() and not any(x in k for x in exclude) and v.shape == model.state_dict()[k].shape}
model.load_state_dict(intersect_state_dict, strict=False)
model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

# reparametrized YOLOR
for i in range((model.nc+5)*anchors):  
    model.state_dict()['model.108.m.0.weight'].data[i, :, :, :] *= state_dict['model.108.im.0.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.108.m.1.weight'].data[i, :, :, :] *= state_dict['model.108.im.1.implicit'].data[:, i, : :].squeeze()
    model.state_dict()['model.108.m.2.weight'].data[i, :, :, :] *= state_dict['model.108.im.2.implicit'].data[:, i, : :].squeeze()
model.state_dict()['model.108.m.0.bias'].data += state_dict['model.108.m.0.weight'].mul(state_dict['model.108.ia.0.implicit']).sum(1).squeeze()
model.state_dict()['model.108.m.1.bias'].data += state_dict['model.108.m.1.weight'].mul(state_dict['model.108.ia.1.implicit']).sum(1).squeeze()
model.state_dict()['model.108.m.2.bias'].data += state_dict['model.108.m.2.weight'].mul(state_dict['model.108.ia.2.implicit']).sum(1).squeeze()
model.state_dict()['model.108.m.0.bias'].data *= state_dict['model.108.im.0.implicit'].data.squeeze()
model.state_dict()['model.108.m.1.bias'].data *= state_dict['model.108.im.1.implicit'].data.squeeze()
model.state_dict()['model.108.m.2.bias'].data *= state_dict['model.108.im.2.implicit'].data.squeeze()

# model to be saved
ckpt = {'model': deepcopy(model.module if is_parallel(model) else model).half(),
        'optimizer': None,
        'training_results': None,
        'epoch': -1}

# save reparameterized model
torch.save(ckpt, save_path)

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


##如果使用原本YOLOv7中的detect.py去做Ensemble將會出錯

In [ ]:
!python detect.py --weights yolov7-prb_final.pt yolov7-prb_final_640.pt --img 640 --conf-thres 0.6 --iou-thres 0.5 --source /content/drive/MyDrive/public
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.6, device='', exist_ok=False, img_size=640, iou_thres=0.5, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='/content/drive/MyDrive/public', update=False, view_img=False, weights=['yolov7-prb_final.pt', 'yolov7-prb_final_640.pt'])
YOLOR 🚀 4d5304c torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 887 layers, 100706449 parameters, 12436561 gradients, 251.5 GFLOPS
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_re

##已更改detect.py，將Ensemble Learning 實現在detect.py中

In [ ]:
!python detect.py --weights best_1920.pt weight/e6e-best.pt --img 1920 --source /content/drive/MyDrive/public --augment

Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=1920, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='/content/drive/MyDrive/public', update=False, view_img=False, weights=['best_1920.pt', 'weight/e6e-best.pt'])
YOLOR 🚀 4d5304c torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
/usr/local/lib/python3.8/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 887 layers, 100706449 parameters, 12436561 gradients, 251.5 GFLOPS
Fusing layers... 
Model Summary: 792 layers, 150956908 parameters, 86508 gradients, 208.9 GFLOP

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec  3 12:42:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------